# Instruct tuning the model

This notebook draws heavily a similar one done for the [phi3](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct/blob/main/sample_finetune.py) model. 

The difference here is that this will focus on a model's full fine-tuning process, work for going from a base model to a new insruction model, and should work for almost any model on HuggingFace.

At the end of the notebook are the steps to save this as a gguf format which will allow for fast and easy inference.

In [1]:
import sys
import logging

import datasets
from datasets import load_dataset
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import os
import json
import wandb


In [10]:


####################
# Base Model Loading
####################
checkpoint_path = "HuggingFaceTB/SmolLM2-360M-Instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
#    attn_implementation="flash_attention_2",  # only works on latest gpus, probably not worth it in most cases
     torch_dtype=torch.bfloat16,
   device_map='auto'
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)


###################
# Tokenizer Loading
###################

checkpoint_path = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = "<|endoftext|>"  # note this is specific to smollm
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token )
tokenizer.padding_side = 'right'
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr


In [14]:
model.eval();
prompt = """I have 45 pills. Sofie dose is 1 pill in morning and half pill at night. How long will this last"""
system_prompt = "<|im_start|>system\nYou are Pi-Card, the Raspberry Pi voice assistant.<|im_end|>\n"
prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

<|im_start|>user
I have 45 pills. Sofie dose is 1 pill in morning and half pill at night. How long will this last<|im_end|>
<|im_start|>assistant
To determine how long the medication will last, we need to calculate the total dose and then divide it by the dose Sophie takes at night.

The medication is 45 pills, and Sophie takes half of it at night. So, Sophie takes 45/2 = 22.5 pills at night.

Now, we need to divide the total dose by Sophie's dose at night. The total dose is 45 pills, and Sophie takes 22.5 pills at night, so the total dose is 45/22.5 = 1.96 pills.

Since we can't have a fraction of a pill, we'll round down to the nearest whole number. However, since Sophie takes half of the total dose, we can't have exactly half of a pill. But we can say that the medication will last approximately 1.96 pills.

So, the medication will last approximately 1.96 pills.<|im_end|><|im_end|>


In [6]:
print(output_text)

<|im_start|>system
You are Pi-Card, the Raspberry Pi voice assistant.<|im_end|>
<|im_start|>user
I have 45 pills. Sofie dose is 1 pill in morning and half pill at night. How long will this last<|im_end|>
<|im_start|>assistant
Sofie's daily dose is a 1500mg pill that can be broken into 4 parts. You are left with 45 pills. So you will have 45 pills left. To know how long their "pill" will last, you can divide 45 pills by the total dose of 1500mg and then divide 1 by 4 parts, which is then dividing 1 by 4. 

However, a more precise measure of 1500mg is in milligrams. The dose was 1 pill in morning and half a pill at night, so in terms of a milligram of medication, this is half of the 1500mg pill. Then to divide a half of 1500mg by the number of doses Sofie takes in a day, which gives you 1500 divided by half, that is 750mg each dose, divided by 2, you get a divide of 375mg per dose, but that's per dose, so it's half of that if that was the answer 375 per dose. But it actually is halved of